In [ ]:
sim = '1100g'

In [ ]:
############################################## SETTINGS ##############################################  
import os
from os import walk
from tqdm import tqdm
import pickle
import random 
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import math
import colorednoise as cn
import datetime 

import seaborn as sns
import matplotlib as mpl

from plotly.subplots import make_subplots
import copy

# Neuron
from neuron import h
import neuron as nrn
nrn.load_mechanisms('mechanisms/single')

# import custom functions
from master_functions import *
#  Setup neuron
h.load_file('stdlib.hoc')
h.load_file('import3d.hoc')

method = 1                  # if 0 stimulate away from soma; if 1 then stimulate towards
physiological = True        # if False then uses original values else more physiological settings for phasic conductances 
                            # if model = 0 then original cell; if model = 1 then paper 
model = 1
cell_type='dspn'
specs = {'dspn': {
                    'N': 71,
                    'lib': 'Libraries/D1_71bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-dMSN_P270-20_1.02_SGA1-m24.swc',
                    },
         'ispn': {
                    'N': 34,
                    'lib': 'Libraries/D2_34bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-iMSN_P270-09_1.01_SGA2-m1.swc'}
        }


specs[cell_type]['model'] = model
morphology = specs[cell_type]['morph']
if model == 0:
    method = 0    
if model == 2:
    v_init = -84
else:
    v_init = -83.7155
    
frequency = 2000 # determines nseg
d_lambda = 0.05  # determines nseg

dend2remove = None # removes any dendrite and its children from the cell; dend2remove = ['dend[42]', 'dend[0]']

# default settings
current_step = False
if current_step:
    sim_time = 1000 
    step_end = sim_time - 100
    step_duration = 500
    step_start = step_end - step_duration
    step_current = -200
else:
    sim_time = 400 

stim_time = 150
baseline = 20

gaba_locations = None      # if None then default placement is the midpoint of the dendrite

timing_range = None

add_noise = False
beta=0                     # if add_noise is True then 0 is white noise
B=1                        # if add_noise is True then scaling factor B=1

add_sine = False 
axoshaft = False

current_step = False
step_current = -200
step_duration = 500
holding_current = 0
Cm = 1
Ra = 200
spine_per_length = 1.61
spine_neck_diam = 0.1
spine_neck_len = 1
spine_head_diam = 0.5
space_clamp = False
record_dendrite = None 
record_location = None 
record_currents = False     # for synaptic currents GABA and glutamatergic
record_branch = False       # if True then also determine voltages in all unique sections of that branch
dend_glut2 = None

record_mechs = False
record_path_dend = False    # if True then calculates voltages and i mechanisms (if record_mechs = True) in unique sections of dendrites within pathlist
record_path_spines = False  # if True then calculates voltages and i mechanisms (if record_mechs = True) in unique spines within pathlist
record_all_path = True      # record at all unique points in the path (including points beyond site of activation)
record_3D = False           # record voltages at all unique segments of every section
record_3D_impedance = False # record voltages at all unique segments of every section
freq = 10                   # impedance measures made at 10Hz
record_3D_mechs = False     # record mechanisms at all unique segments of every section
record_Ca = False
record_3D_Ca = False
tonic = False               # add tonic GABA conductance
gbar_gaba = None            # add gbar for tonic GABA
rectification = False       # if tonic GABA then choose whether it is recitified or not
distributed = False         # can specify the distribution of tonic GABA using GABA params
gaba_params = None
tonic_gaba_reversal = -60

dt = 0.025 # 1 

space_clamp = False
show_figs = True    

In [ ]:
# if these FIXED VARIABLES exist then record them
# keep adding as required
variable_names = [
     'AMPA',
     'axoshaft',
     'axospine'
     'cell_coordinates',
     'cell_type',
     'model',
     'physiological',
     'NMDA',
     'dend_gaba',
     'dend_glut',
     'dt',
     'Ndend_gaba',
     'Nsim_plot',
     'Nsim_save',
     'Nsims',
     'add_noise',
     'burn_time',
     'current_step',
     'dend_gaba',
     'dend_glut',
     'freq',
     'g_AMPA',
     'g_GABA',
     'gaba_frequency',
     'gaba_locations',
     'gaba_locs',
     'gaba_range',
     'gaba_reversal',
     'glut_frequency',
     'glutamate_locations',
     'glutamate_locs',
     'glut_range',
     'holding_current',
     'impedance',
     'num_gabas',
     'num_gluts',
     'ratio',
     'record_dendrite',
     'record_dends',
     'record_dists',
     'record_location',
     'record_locs',
     'rel_gaba_onsets',
     'save',
     'showplot',
     'sim',
     'sim_time',
     'space_clamp',
     'spine_per_length',
     'spine_neck_diam',
     'spine_neck_len',
     'spine_head_diam',
     'start_time',
     'stim_time',
     'timing_range',
     'vary_gaba_location',
     'vary_gaba_time',
     'vary_location',
     'dend2remove',
     'tonic',
     'gbar_gaba'
]


In [ ]:
############################################### sim 1100 ###############################################   
sims1100 = ['1100' + letter for letter in 'abcdefgh']
if check_sim(sim, sims1100):
    
    change_Ra = False
    if change_Ra:
        Ra = 350
    
    record_3D = True
    record_3D_mechs = True
    record_3D_Ca = True
    record_Ca = False
    record_currents = True
    record_branch = True 
    record_mechs = True
    record_path_dend = True 
    record_path_spines = True     
    
    # show overall summary plot and save
    showplot = True; 
    save = True
    
    # (nearest synapse to relative location is selected)
    axoshaft = False # if True then places glutamatergic synapse on the shaft
    
    # if rec_all_path is True then record all voltages along the path (proximal and distal) to synapse of interest

    glut = True
    
    glut_frequency = 1000 # every 1ms
    num_gluts_dict = {
        '1100a': 1, '1100b': 3,
        '1100c': 6, '1100d': 9,
        '1100e': 12, '1100f': 15,
        '1100g': 18, '1100h': 21,
    }
    
    if glut:
        num_gluts = num_gluts_dict.get(sim, 'Default Value')
    else:
        num_gluts = 0
    
    dend_glut = ['dend[15]']
    
    if record_branch:
        dend_glut2 = ['dend[8]', 'dend[10]', 'dend[12]', 'dend[14]']
    
    glutamate_locations = spine_locator(cell_type=cell_type, specs=specs, spine_per_length=spine_per_length, frequency=frequency, d_lambda=d_lambda, dend_glut=dend_glut, num_gluts=num_gluts)
 
    record_dendrite = dend_glut[0]
    record_location = [0.5] # None
    
    g_AMPA = 0.001 
    AMPA = True
    NMDA = True
    ratio = 2.15 # NMDA to AMPA conductance ratio (not necessarily the current ratio)

    # Phasic gaba
    gaba = False
    # gaba_locations = None # if None then default placement is the midpoint of the dendrite
    gaba_locations = [0.5] 
    
    num_gabas = 12 # number of GABA synapses in total
    if not gaba:
        num_gabas = 0
        
    # specify dend_gaba locations correctly
    dend_gaba = ['dend[15]'] *  8
    dend_gaba = ['dend[28]', 'dend[46]', 'dend[36]', 'dend[57]'] * int(num_gabas/4)

    # record_dendrite must be site of GABA synapse
    #     1. either specify rel_gaba_onsets
    #     2. or if gaba_frequency = None then ALL AT SAME TIME or at specified frequency eg gaba_frequency = 100
    gaba_frequency = None 
    
    Ndend_gaba = count_unique_dends(dend_gaba)
    rel_gaba_onsets = rel_gaba_onset(n=num_gabas, N=Ndend_gaba)
    g_GABA = 0.001
    
    gaba_reversal = -60
    vary_gaba_time = False # if true timing of gaba is varied relative to glut; if false then vary glut relative to gaba

    # timing_range  = list(range(120, 261, 1))
    timing_range = list(range(160,170,10)) # range(140, 150, 10) # 200    
    gaba_range = np.repeat(False, len(timing_range)) 
    glut_range = np.repeat(True, len(timing_range)) 
    
    # each sim will produce some plots; turn off if large number of sims
    Nsim_save = False
    Nsim_plot = True
    
    Nsims = len(timing_range)

In [ ]:
# to store sim-generated variables
data_dict = {
    'v_all_3D': {},
    'Ca_all_3D': {},
    'imp_all_3D': {},
    'i_mechs_3D': {},
    'v_dend_tree': {},
    'v_spine_tree': {},
    'v_branch': {},
    'vsoma': [],
    'vdend': [],
    'v_dend_activated': {},
    'vspine': [],
    'v_spine_activated': {},
    'Ca_soma': [],
    'Ca_dend': [],
    'Ca_spine': [],
    'timing': [],
    'dists': [],
    'dists_spine': [],
    'i_mechs_dend': {},
    'i_mechs_dend_tree': {},
    'i_mechs_spine_tree': {},
    'i_ampa': {},
    'i_nmda': {},
    'i_gaba': {},
    'g_gaba': {},
    'record_dists':[],
    'record_spine':[],
    'spine_dist': [],
    'capacitance': [],
    'timestamp': {}
    }

# get coordinates for all unique segments within all sections
cell, spines, dend_tree = cell_build(cell_type=cell_type, specs=specs, addSpines=True, spine_per_length=spine_per_length, frequency=frequency, d_lambda=d_lambda, verbose=False, dend2remove=dend2remove)
mechs=['pas', 'kdr', 'naf', 'kaf', 'kas', 'kir', 'cal12', 'cal13', 'can', 'car', 'cav32', 'cav33', 'sk', 'bk']
mech_distr_3D = record_mechs_distr_3D(cell=cell, mechs=mechs)
        
# set these to False is they are not already assigned
glut_reversal = globals().get('glut_reversal', 0) 
vary_location = globals().get('vary_location', False)  
dend_glut_range = globals().get('dend_glut_range', None)  
vary_gaba_location = globals().get('vary_gaba_location', False)   
    
# rec_all_path = globals().get('rec_all_path', False)  # default is False
# if False records all v from spine / dendrite to soma
# if True records at all unique sites including those distal to the synapse

# determine if axoshaft or axospinous glutamatergic synapse
axoshaft = globals().get('axoshaft', False)
axospine = not axoshaft

if model == 1 and not vary_location:
    glutamate_locations = sorted(glutamate_locations, reverse=True)

metadata = {}
# Add fixed variables to metadata
for name in variable_names:
    try:
        if name not in metadata:
            metadata[name] = eval(name)
    except NameError:
#         print(f'Variable {name} not found!') # no need to print not found variables
        continue

# syn_reversals(cell_type, specs, spine_per_length, frequency, d_lambda, dend_glut, glut_reversal, glutamate_locations, num_gluts, dend_gaba, gaba_reversal, gaba_locations, num_gabas, sim_time, dt=0.025, v_init=-84)

# Common parameters
common_params = {
    'cell_type': cell_type, 
    'specs': specs, 
    'spine_per_length':spine_per_length,
    'frequency': frequency,
    'd_lambda': d_lambda,
    'glut_reversal': glut_reversal,
    'num_gluts': num_gluts,
    'gaba_reversal': gaba_reversal,
    'num_gabas': num_gabas,
    'sim_time': stim_time,
    'dt': dt,
    'v_init': v_init,
    'dend2remove': dend2remove,
}

if not vary_location:
    syn_reversals_params = {
        'dend_glut': dend_glut,
        'glutamate_locations': glutamate_locations,
        'dend_gaba': dend_gaba,
        'gaba_locations': gaba_locations,
        **common_params
    }
    glut_reversals, gaba_reversals = syn_reversals(**syn_reversals_params)

else:
    syn_reversals_params = {
        **common_params
    }
    if vary_gaba_location:
        syn_reversals_params.update({
            'dend_glut': dend_glut,
            'glutamate_locations': glut_locations,
            'dend_gaba': dend_gaba_range,
            'gaba_locations': gaba_locations_range,
        })
        glut_reversals, gaba_reversals_range = syn_reversals(**syn_reversals_params)
        gaba_reversals_range = gaba_reversals_range * len(gaba_locations_range)
    else:
        syn_reversals_params.update({
            'dend_glut': dend_glut_range,
            'glutamate_locations': glut_locations_range,
            'dend_gaba': dend_gaba,
            'gaba_locations': gaba_locations,
        })
        glut_reversals_range, gaba_reversals = syn_reversals(**syn_reversals_params)
        glut_reversals_range = glut_reversals_range * len(glut_locations_range)
        
# perform Nsim simulations 
for ii in tqdm(range(Nsims)): # for ii in tqdm(list(range(0,6))):
    # time stamp date
    time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

    sim_image_path = None
    if Nsim_save:
        sim_image_path = '{}/model{}/{}/images/sim{}/Nsim{}'.format(cell_type, model, 'physiological' if physiological else 'nonphysiological', sim, ii)
        os.makedirs(sim_image_path, exist_ok=True)

    # set variables for this simulation
    timing = timing_range[ii]
    gaba = gaba_range[ii]
    glut = glut_range[ii]
    if vary_gaba_time:
        glut_time = stim_time
        gaba_time = timing
    else:
        glut_time = timing
        gaba_time = stim_time
    if vary_location:
        if vary_gaba_location: 
            dend_gaba = [dend_gaba_range[ii]]
            gaba_reversals = [gaba_reversals_range[ii]] if num_gabas == 1 else gaba_reversals_range[ii]

        else:
            dend_glut = [dend_glut_range[ii]]
            glut_reversals = [glut_reversals_range[ii]] if num_gluts == 1 else glut_reversals_range[ii]

        record_dendrite = record_dends[ii]
        glutamate_locations = [glut_locations_range[ii]]
        gaba_locations = [gaba_locations_range[ii]]
        record_location = [record_locs[ii]]

    if tonic:
        gbar_gaba = gbar_gaba_range[ii]
        print('tonic GABA ', gbar_gaba)
    else:
        gbar_gaba = 0
    
    protocol = 'Nsim{}'.format(ii)

    # Run sim           
    v_all_3D, Ca_all_3D, imp_all_3D, i_mechs_3D, vspine, v_spine_activated, vdend, \
    v_dend_activated, vsoma, v_dend_tree, v_spine_tree, Ca_spine, Ca_dend, Ca_soma, \
    i_mechs_dend, i_mechs_dend_tree, i_mechs_spine_tree, v_branch, zdend, ztransfer, \
    ampa_currents, nmda_currents, gaba_currents, gaba_conductances, record_dist, \
    record_spine, spine_dist, cap, dt, burn_time, start_time = \
    CurrentClamp(sim_time=sim_time, 
                    stim_time=stim_time,
                    baseline=baseline,
                    glut=glut, 
                    glut_frequency=glut_frequency, 
                    glutamate_locations=glutamate_locations, 
                    glut_reversals=glut_reversals,
                    glut_time=glut_time, 
                    num_gluts=num_gluts, 
                    dend_glut=dend_glut, 
                    g_AMPA=g_AMPA,
                    ratio=ratio,
                    gaba=gaba, 
                    gaba_frequency=gaba_frequency, 
                    gaba_locations=gaba_locations, 
                    gaba_reversals=gaba_reversals,
                    gaba_time=gaba_time, 
                    g_GABA=g_GABA, 
                    num_gabas=num_gabas, 
                    dend_gaba=dend_gaba, 
                    specs=specs, 
                    frequency=frequency,
                    d_lambda=d_lambda,
                    dend2remove=dend2remove,
                    v_init=v_init,
                    AMPA=AMPA,
                    NMDA=NMDA,
                    method=method,
                    physiological=physiological,
                    timing_range=timing_range,
                    add_noise=add_noise,
                    beta=beta,                   
                    B=B,                      
                    add_sine=add_sine, 
                    axoshaft=axoshaft,
                    cell_type=cell_type,
                    current_step=current_step,
                    step_current=step_current,
                    step_duration=step_duration,
                    holding_current=holding_current,
                    Cm=Cm,
                    Ra=Ra,
                    spine_per_length=spine_per_length,
                    spine_neck_diam=spine_neck_diam,
                    spine_neck_len=spine_neck_len,
                    spine_head_diam=spine_head_diam,
                    space_clamp=space_clamp,
                    record_dendrite=record_dendrite, 
                    record_location=record_location, 
                    record_currents=record_currents,
                    record_branch=record_branch,
                    dend_glut2=dend_glut2, 
                    record_mechs=record_mechs,
                    record_path_dend=record_path_dend,   
                    record_path_spines=record_path_spines,  
                    record_all_path=record_all_path,
                    record_3D=record_3D,         
                    record_3D_impedance=record_3D_impedance,
                    freq=freq,                   
                    record_3D_mechs=record_3D_mechs,    
                    record_Ca=record_Ca,
                    record_3D_Ca=record_3D_Ca,
                    tonic=tonic,
                    gbar_gaba=gbar_gaba,
                    rectification=rectification,       
                    distributed=distributed,         
                    gaba_params=gaba_params,
                    tonic_gaba_reversal=tonic_gaba_reversal,
                    dt=dt
                    )
        
    # only do if want to view each sim or save sim graphs
    if record_path_dend:
        plots_return(v_tree=v_dend_tree['v'], vspine=vspine, dists=v_dend_tree['dists'], spine_dist=spine_dist, num_gluts=num_gluts, start_time=start_time, burn_time=burn_time, dt=dt, xaxis_range=[0,200], Nsim_plot=Nsim_plot, Nsim_save=Nsim_save, sim_image_path=sim_image_path, time=time)

    update_data_dictionary(data_dict=data_dict, protocol=protocol, v_all_3D=v_all_3D, 
            Ca_all_3D=Ca_all_3D, imp_all_3D=imp_all_3D, i_mechs_3D=i_mechs_3D, 
            vspine=vspine, v_spine_activated=v_spine_activated, vdend=vdend, v_dend_activated=v_dend_activated, 
            vsoma=vsoma, v_dend_tree=v_dend_tree, v_spine_tree=v_spine_tree, Ca_spine=Ca_spine, 
            Ca_dend=Ca_dend, Ca_soma=Ca_soma, timing=timing, i_mechs_dend=i_mechs_dend, 
            i_mechs_dend_tree=i_mechs_dend_tree, i_mechs_spine_tree=i_mechs_spine_tree, v_branch=v_branch, 
            ampa_currents=ampa_currents, nmda_currents=nmda_currents, gaba_currents=gaba_currents, 
            gaba_conductances=gaba_conductances, record_dist=record_dist, time=time, 
            record_currents=record_currents, record_spine=record_spine, spine_dist=spine_dist, cap=cap)

data_dict['metadata'] = metadata
data_dict['metadata']['dt'] = dt
data_dict['mechanisms_3D_distribution'] = mech_distr_3D

data_dict['metadata']['spine_per_length'] = spine_per_length
data_dict['metadata']['spine_neck_diam'] = spine_neck_diam
data_dict['metadata']['spine_neck_len'] = spine_neck_len
data_dict['metadata']['spine_head_diam'] = spine_head_diam

# Save
if save:
    simulations_path = '{}/model{}/{}/simulations/sim{}'.format(cell_type, model, 'physiological' if physiological else 'nonphysiological', sim)
    os.makedirs(simulations_path, exist_ok=True)
    names = list(data_dict.keys())
    for name in names:
        with open('{}/{}.pickle'.format(simulations_path, name), 'wb') as handle:
            pickle.dump(data_dict[name], handle, protocol=pickle.HIGHEST_PROTOCOL)  
